In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
df = [train,test]
df_combined = pd.concat(df).reset_index(drop=True)


In [ ]:
f4th_label = df_combined.iloc[3, 0]
f4th_image = df_combined.iloc[3, 1:].values.reshape(28, 28)

plt.imshow(f4th_image, cmap='gray')
plt.title(f"Label: {f4th_label}")
plt.axis('off')
plt.show()

In [ ]:
X = pd.DataFrame(df_combined[:42000])
X_Test  = pd.DataFrame(df_combined[42000:])
y = train['label']

In [ ]:
X_Test.drop(columns=['label'],inplace=True)

In [ ]:
X.drop(columns=['label'],inplace=True)
y.drop(columns=['label'], inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.values
X_test = X_test.values
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)


svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_flat, y_train)


svm_preds = svm_model.predict(X_test_flat)


svm_accuracy = accuracy_score(y_test, svm_preds)
print(f'SVM Test Accuracy: {svm_accuracy}')

In [ ]:

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_flat, y_train)


knn_preds = knn_model.predict(X_test_flat)

knn_accuracy = accuracy_score(y_test, knn_preds)
print(f'k-NN Test Accuracy: {knn_accuracy}')

**CNN**

In [ ]:
X_train = X_train.values
X_test = X_test.values
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=6, batch_size=64, validation_split=0.2)


test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
X_test = X_Test.values


X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0


predictions = model.predict(X_test)


predicted_labels = np.argmax(predictions, axis=1)

output_df = pd.DataFrame({'ImageId': range(1, len(X_Test) + 1), 'label': predicted_labels})

output_df.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")